# Agenda

1. Dates and times
2. Visualization
3. Optimization of queries
    - Memory usage (dtypes and categories)
    - Techniques for improving query speed
    - PyArrow

# Dates and times

When we use the word "time" in a human language, we actually mean two different things:

- A specific point in time, unique in the universe's history. We point to that time with a particular year, month, date, hour, minute, second, etc., depending on how fine-grained we want our measurement to be.  This is how we indicate when a class starts, or when a meeting ends, or when someone was born. 